In [45]:
import pandas as pd
import umap
import numpy as np

In [31]:
path = r'D:\github\2dv50e\Data\1. Heart Disease'
df_probabilities = pd.read_csv(path + r'\topModelsProbabilities.csv')
df_model = pd.read_csv(path + r'\topModels.csv')
algo_nr = df_model['algorithm_id']

algos = {1:'K-Nearest Neighbor', 2:'Support Vector Machine', 3:'Gaussian Naive Bayes', 4:'Multilayer Perceptron', 5:'Logistic Regression',
        6:'Linear Discriminant Analysis', 7:'Quadratic Discriminant Analysis', 8:'Random Forest', 9:'Extra Trees', 10:'Adaptive Boosting',
        11:'Gradient Boosting'}

In [32]:
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(df_probabilities)

In [35]:
#convert umap_embedding to dataframe
df_umap = pd.DataFrame(umap_embedding, columns=['UMAP_1', 'UMAP_2'])
# Add model name
df_umap['algorithm_nr'] = algo_nr

In [36]:
df_umap.head()

,UMAP_1,UMAP_2,algorithm_nr
0,7.046518,9.144990,1
1,6.706904,9.036844,1
2,6.815217,9.084249,1
3,6.942645,8.775805,1
4,6.847733,8.640591,1


In [37]:
df = df_umap.copy()

In [55]:
df['algorithm_name'] = df['algorithm_nr'].map(algos)
df['performance'] = df_model['overall_performance']/100
df.head()

,UMAP_1,UMAP_2,algorithm_nr,algorithm_name,performance
0,7.046518,9.144990,1,K-Nearest Neighbor,0.7293
1,6.706904,9.036844,1,K-Nearest Neighbor,0.7287
2,6.815217,9.084249,1,K-Nearest Neighbor,0.7274
3,6.942645,8.775805,1,K-Nearest Neighbor,0.7273
4,6.847733,8.640591,1,K-Nearest Neighbor,0.7272


In [40]:
type(df.algorithm_name.unique())

numpy.ndarray

In [42]:
algos.values()

dict_values(['K-Nearest Neighbor', 'Support Vector Machine', 'Gaussian Naive Bayes', 'Multilayer Perceptron', 'Logistic Regression', 'Linear Discriminant Analysis', 'Quadratic Discriminant Analysis', 'Random Forest', 'Extra Trees', 'Adaptive Boosting', 'Gradient Boosting'])

In [46]:
algos_unique = np.unique(algos.values())

In [47]:
algos_unique

array([dict_values(['K-Nearest Neighbor', 'Support Vector Machine', 'Gaussian Naive Bayes', 'Multilayer Perceptron', 'Logistic Regression', 'Linear Discriminant Analysis', 'Quadratic Discriminant Analysis', 'Random Forest', 'Extra Trees', 'Adaptive Boosting', 'Gradient Boosting'])],
      dtype=object)

In [48]:
d = dict(zip(algos_unique, np.arange(len(algos_unique))))

In [49]:
d

{dict_values(['K-Nearest Neighbor', 'Support Vector Machine', 'Gaussian Naive Bayes', 'Multilayer Perceptron', 'Logistic Regression', 'Linear Discriminant Analysis', 'Quadratic Discriminant Analysis', 'Random Forest', 'Extra Trees', 'Adaptive Boosting', 'Gradient Boosting']): 0}

In [54]:
algos.values()

dict_values(['K-Nearest Neighbor', 'Support Vector Machine', 'Gaussian Naive Bayes', 'Multilayer Perceptron', 'Logistic Regression', 'Linear Discriminant Analysis', 'Quadratic Discriminant Analysis', 'Random Forest', 'Extra Trees', 'Adaptive Boosting', 'Gradient Boosting'])